In [1]:
%run "preprocess_data.py"

question = "Does the charter allow or require indemnification of directors for liability?"

filepath = "data/ClassificationData.xlsx"
sheet_name = "IndemnificationFeeInsurance"
df = import_data_from_excel(filepath, sheet_name)
df = df[df.apply(lambda x: x["Question"] == question, axis=1)]

### apply the classifier to a hand-curated data set

In [35]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

cls_model = AutoModelForSequenceClassification.from_pretrained("./qa/1657115593/models\checkpoint-69")
cls_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
classifier = pipeline("text-classification", model=cls_model, tokenizer=cls_tokenizer)

for idx, row in df.iterrows():
    context = row["Context"]
    pred = classifier(context)[0]
    label = "Required" if pred["label"] == "LABEL_2" else "Allow" if pred["label"] == "LABEL_1" else "N"
    score = str(pred["score"])[:4]
    print("label: {}\nscore: {}\ncontext: {}\n".format(label, score, context))

label: Allow
score: 0.89
context: The Company is authorized to indemnify to the fullest extent permitted by law any person made or threatened to be made a party to an action or proceeding, whether criminal, civil, administrative or investigative, by reason of the fact that such person was a director or officer of the Company or any predecessor of the Company, or serves or served at any other enterprise as a director or officer at the request of the Company or any predecessor to the Company.

label: Allow
score: 0.97
context: To the fullest extent permitted by applicable law, the Corporation is authorized to provide indemnification of (and advancement of expenses to) directors, officers and agents of the Corporation (and any other persons to which General Corporation Law permits the Corporation to provide indemnification) through Bylaw provisions, agreements with such agents or other persons, vote of stockholders or disinterested directors or otherwise, in excess of the indemnification 

### try to apply the classifier on every clause of a corporate charter

In [ ]:
%run "scan_file.py"
%run "text_tree.py"

from sentence_transformers import SentenceTransformer, util
sts_transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def iter_classify(cls, sts, node):
    if node.text:
        context = node.text
        context_embedding = sts.encode(context)
        question_embedding = sts.encode(question)
        similarity_score = util.pytorch_cos_sim(question_embedding, context_embedding)[0][0].item()
        if similarity_score > 0.5:
            pred = cls(context)[0]
            label = "Required" if pred["label"] == "LABEL_2" else "Allow" if pred["label"] == "LABEL_1" else "N"
            score = str(pred["score"])[:4]
            print("label: {}\nscore: {}\ncontext: {}\n".format(label, score, context))

    for child in node.child_nodes:
        iter_classify(cls, sts, child)

In [36]:
pdf = scan_pdf("indemnification_required.pdf")
tree = build_text_tree(pdf)
iter_classify(classifier, sts_transformer, tree)

label: N
score: 0.98
context: (b) Notwithstanding the provisions of Section 223(a)(l) and 223(a)(2) of the Delaware General Corporation Law, any vacancy, including newly created directorships resulting from any increase in the authorized number of directors or amendment of this Restated Certificate, and vacancies created by removal or resignation of a director, may be filled by a majority of the directors then in office, though less than a quorum, or by a sole remaining director, and the directors so chosen shall hold office until the next annual election and until their successors are duly elected and shall qualify, unless sooner displaced; provided, however, that where such vacancy occurs among the directors elected by the holders of a class or series of stock, the holders of shares of such class or series may override the Board of Directors' action to fill such vacancy by (i) voting for their own designee to fill such vacancy at a meeting of the Corporation's stockholders or (ii) wr

In [37]:
pdf2 = scan_pdf("fee_advancement_allowed.pdf")
tree2 = build_text_tree(pdf2)
iter_classify(classifier, sts_transformer, tree2)

label: N
score: 0.98
context: A. LIMITATION. To the fullest extent permitted by law, a director of the Corporation shall not be personally liable to the Corporation or its stockholders for monetary damages for breach of fiduciary duty as a director. If the General Corporation Law or any otber law of the State of Delaware is amended after approval by tbe stockholders of this Article VIII to autborize corporate action further eliminating or limiting tbe personal liability of directors, then the liability of a director of tbe Corporation shaH be eliminated or limited to the fullest extent permitted by the General Corporation Law as so amended. Any repeal or modification of the foregoing provisions of this Article VIII by the stockholders of tbe Corporation shall not adversely affect any right or protection of a director of tbe Corporation existing at the time of, or increase the liability of any director of tbe Corporation with rcspect to any acts or omissions of su(;h director occurring 

In [39]:
pdf3 = scan_pdf("named_docs/DE_Unequal_CommonStocks.pdf")
tree3 = build_text_tree(pdf3)
iter_classify(classifier, sts_transformer, tree3)

an OCR-scanned file
label: N
score: 0.98
context: (2) insofar as it consists of property other than cash, be computed at the fair market value thereof at the time of such issuance, as determined in good faith by the Board of Directors, including the approval of at least one of the Preferred Directors; and

label: N
score: 0.98
context: ARTICLE VI. Except as otherwise provided in this Restated Certificate, in furtherance and not in limitation of the powers conferred by statute, the Board of Directors is expressly authorized to make, alter, amend or repeal the bylaws of the Company.

label: N
score: 0.98
context: ARTICLE VIII. To the fullest extent permitted by the Delaware General Corporation Law, as the same exists or as may hereafter be amended from time to time, a director of the Company shall not be personally liable to the Company or its stockholders for monetary damages for breach of fiduciary duty as a director. If the Delaware General Corporation Law is amended to authorize corp

In [40]:
pdf4 = scan_pdf("named_docs/CA_Restatement.pdf")
tree4 = build_text_tree(pdf4)
iter_classify(classifier, sts_transformer, tree4)

an OCR-scanned file
label: N
score: 0.97
context: A. Limitation of Directors" Liability. The liability of the directors of the corporation for monetary damages shall be eliminated to the fullest extent permissible under California law. A0755685

label: Allow
score: 0.70
context: B. Indemnification of Corporate Agents. The corporation is authorized to provide indemnification of agents (as defined in Section 317 of the California Corporations Code) through bylaw provisions, agreements with agents, vote of shareholders or disinterested directors or otherwise, in excess of the indemnification otherwise permitted by Section 317 of the California Corporations Code, subject only to the applicable limits set forth in Section 204 of the California Corporations Code with respect to actions for breach of duty to the corporation and its shareholders. Cc. Repeal or Modification. No repeal or modification of this article shall adversely affect any right to limitation of liability of a director or in